# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: CHETAN YADAV VENKATESH
#### Student ID: s4137946


Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment
* pandas - for reading and handling tabular datasets
* re - for applying refulaer expressions during tokenization
* numpy - for numerical operations and arrays
* collections.Counter - to count word frequencies across the corpus
* string - for handling text cleaning if needed

## Introduction
In this milestone, we are working with a dataset of women’s clothing reviews.
Our task is to pre-process the review texts and prepare them for later use in machine learning models.

Why is this important?

Raw customer reviews often contain noise such as stopwords, rare words, or overly frequent terms.

Pre-processing ensures that the text data is clean, consistent, and meaningful.

Finally, we will generate a vocabulary file (vocab.txt) which maps each unique token to an integer index. This vocabulary will be the backbone for encoding reviews into machine-readable form.

Text preprocessing transforms raw, noisy customer reviews into clean, structured data suitable for machine learning. Raw reviews contain inconsistencies—"LOVE", "love", and "loved" appear as different words; punctuation and stopwords add noise without meaning. Proper preprocessing standardizes text, removes noise, and creates a clean vocabulary of meaningful terms.
This task implements a 9-step pipeline: tokenization, lowercase conversion, length filtering, stopword removal, rare word filtering, frequent word filtering, data export, and vocabulary generation. Each step builds upon the previous to produce a curated 7,529-word vocabulary from 19,662 reviews.

## Importing libraries 

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from pathlib import Path
from typing import List


In [2]:
RAW_CSV = "assignment3.csv"           # given dataset
STOPWORDS_TXT = "stopwords_en.txt"    # provided stopword list
PROCESSED_CSV = "processed.csv"       # required output
VOCAB_TXT = "vocab.txt"               # required output


### 1.1 Examining and loading data
Before performing any pre-processing, it is essential to understand the structure of the dataset we are working with. Good data science practice always starts with exploration. This helps us confirm:

Which columns are present in the dataset.

Which columns are useful for our task.

Whether there are missing values or anomalies we need to handle later.

In this dataset (assignment3.csv), each row represents a customer review of a clothing item. The main columns include:

Clothing ID → Identifier of the product reviewed.

Age → Age of the reviewer.

Title → Short summary provided by the reviewer.

Review Text → Detailed customer feedback (this is our main focus).

Rating → Numerical score given by the reviewer.

Recommended IND → Whether the customer recommended the item (1 = yes, 0 = no).

Positive Feedback Count → How many people found the review helpful.

Division Name, Department Name, Class Name → Product metadata (categories).

For Task 1, we are only interested in “Review Text” because this is the raw natural language input we need to clean and prepare for classification.

In [3]:
try:
    data = pd.read_csv(RAW_CSV)
    print(f"✓ Successfully loaded {len(data)} reviews")
except FileNotFoundError:
    raise FileNotFoundError(f"Cannot find {RAW_CSV}. Please ensure the file is in the correct location.")

✓ Successfully loaded 19662 reviews


Findings:
1. The dataset has both numerical and categorical columns, but for NLP pre-processing we will mainly use Review Text.

2. Some reviews have short titles (like “Flattering shirt”) while others provide detailed comments (in the “Review Text” column).

3. The target column “Recommended IND” is binary (0 = not recommended, 1 = recommended), which will be useful in the later modeling stage.

4. For this milestone, we do not need product IDs, age, or metadata; we focus only on cleaning the review text.

### 1.2 Pre-processing data
Perform the required text pre-processing steps.


Step 1:Loading stopwords

Stopwords (common words like "the", "is", "at") carry little meaning. Using a set provides O(1) lookup time for efficient checking during preprocessing.

In [4]:
# Keep only the columns needed downstream
need_cols = ["Title","Review Text","Recommended IND"]
missing = [c for c in need_cols if c not in data.columns]
if missing:
    raise ValueError(f"Missing required columns in {RAW_CSV}: {missing}")

data = data[need_cols].rename(columns={"Recommended IND":"Recommended"})

# ---- Load stopwords ----
with open(STOPWORDS_TXT, "r", encoding="utf-8") as f:
    STOPWORDS = set(w.strip().lower() for w in f if w.strip())

# ---- Tokenizer per spec ----
TOKEN_RE = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"

Step 2: Define Pre-Processing function

This function applies four transformations in sequence:

Tokenize with regex
Lowercase for consistency
Remove short tokens (<2 chars)
Remove stopwords

Example: "I had such high hopes for this dress" → ['high', 'hopes']

In [5]:
from typing import List
def preprocess_review(text: str) -> List[str]:
    # 1) regex tokenize
    toks = re.findall(TOKEN_RE, str(text))
    # 2) lowercase
    toks = [t.lower() for t in toks]
    # 3) remove short tokens (<2)
    toks = [t for t in toks if len(t) >= 2]
    # 4) remove stopwords
    toks = [t for t in toks if t not in STOPWORDS]
    return toks

Step 3: Apply Pre-Processing

In [6]:
#Apply preprocessing to all reviews

data["tokens"] = data["Review Text"].apply(preprocess_review)

# ---- Global term frequency (TF) ----
tf_counter = Counter()
for toks in data["tokens"]:
    tf_counter.update(toks)

Findings:
Each review has been successfully converted into a list of tokens.

For example:

Raw review: “I had such high hopes for this dress and really wanted it to work…”

Processed tokens: [high, hopes, dress, wanted, work, initially, ...]

This transformation helps us:

1. Remove noise such as stopwords, punctuation, and irrelevant words.

2. Standardize text into lowercased, lemmatized tokens.

3. Prepare the corpus for feature extraction (Bag-of-Words, TF-IDF, etc.).

Step 4: Frequency based filetring

Term Frequency (TF): Counts total word occurrences. Removed 6,734 words appearing only once (likely typos/outliers).
Document Frequency (DF): Counts reviews containing each word. Removed top-20 words by DF:

'love' (6,416 reviews - 32.6%)
'size' (5,888 reviews - 29.9%)
'fit' (5,537 reviews - 28.2%)

These are too universal to be discriminative—everyone mentions them regardless of recommendation.
Final vocabulary: 7,529 words (appearing 2+ times but not in top-20 DF).

In [7]:
# Task 1 - After frequency filtering
# Remove rare words (TF=1) to eliminate likely typos and outliers
# These words appear only once across 19,662 reviews and cannot help the model generalize
rare_words = {w for w, c in tf_counter.items() if c == 1}

# ---- Document frequency (DF) ----
df_counter = Counter()
for toks in data["tokens"]:
    df_counter.update(set(toks))

# Remove top-20 DF words to eliminate overly common domain terms
# Words like 'love', 'size', 'fit' appear in 27-33% of reviews
# They are too universal to discriminate between recommendations
top20_df_words = {w for w, _ in df_counter.most_common(20)}

# ---- Final filter (remove rare + top20 DF) ----
to_remove = rare_words | top20_df_words

def filter_tokens(toks: List[str]) -> List[str]:
    return [t for t in toks if t not in to_remove]

data["final_tokens"] = data["tokens"].apply(filter_tokens)



Step 5: Save Processed reviews

In [8]:
# ---- Save processed.csv (must include tokens for later + labels for Task 3) ----
# Store final_tokens as Python-literal lists for safe ast.literal_eval later
out = data[["Title","Review Text","Recommended","final_tokens"]].copy()
out.to_csv(PROCESSED_CSV, index=False)

# ---- Build vocabulary (alphabetical, zero-based) from final_tokens ONLY ----
vocab_set = set()
for toks in data["final_tokens"]:
    vocab_set.update(toks)

vocab_sorted = sorted(vocab_set)
word2idx = {w:i for i, w in enumerate(vocab_sorted)}

## Saving required outputs
processed.csv contains:

Original Title and Review Text (for reference)
Recommended labels (for Task 3)
final_tokens (preprocessed lists)

vocab.txt format: word:index (alphabetically sorted, zero-indexed)

Example: a-cup:0, abandoned:1, ..., zoom:7528


In [9]:
with open(VOCAB_TXT, "w", encoding="utf-8") as f:
    for w in vocab_sorted:
        f.write(f"{w}:{word2idx[w]}\n")

# ---- Sanity checks (do not modify outputs) ----
print(f"Saved {PROCESSED_CSV} with {len(out)} rows.")
print(f"Saved {VOCAB_TXT} with {len(vocab_sorted)} words.")
print(f"Removed rare (TF==1): {len(rare_words)}; removed top-20 DF words: {len(top20_df_words)}.")

Saved processed.csv with 19662 rows.
Saved vocab.txt with 7529 words.
Removed rare (TF==1): 6734; removed top-20 DF words: 20.


Vocabulary alphabetically sorted 
Zero-based indexing (0 to 7,528) 
Top-20 DF words absent from vocabulary 
6,734 rare words removed

In [10]:
# 1. Vocabulary integrity
assert vocab_sorted == sorted(vocab_sorted), "Vocabulary not alphabetical"
assert len(vocab_sorted) == len(set(vocab_sorted)), "Duplicate tokens found"
assert word2idx[vocab_sorted[0]] == 0, "First index is not 0"
assert word2idx[vocab_sorted[-1]] == len(vocab_sorted)-1, "Last index mismatch"
print(" Vocabulary integrity checks passed")

# 2. Filtering proof
top20_with_df = df_counter.most_common(20)
print("Top-20 by DF (token, DF):", top20_with_df)
tf_eq1_count = sum(1 for _, c in tf_counter.items() if c == 1)
print("Tokens removed with TF==1:", tf_eq1_count)

assert all(w not in word2idx for w, _ in top20_with_df), "Top-20 DF token leaked into vocab"
print(" Filtering checks passed")

# 3. Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print(f"Random state fixed at {RANDOM_STATE}")

 Vocabulary integrity checks passed
Top-20 by DF (token, DF): [('love', 6416), ('size', 5888), ('fit', 5537), ('dress', 5346), ('wear', 4900), ('top', 4670), ('great', 4497), ('fabric', 3712), ('color', 3604), ('small', 3265), ('ordered', 3099), ('perfect', 2973), ('flattering', 2939), ('soft', 2805), ('comfortable', 2597), ('back', 2538), ('cute', 2398), ('fits', 2394), ('nice', 2393), ('bought', 2376)]
Tokens removed with TF==1: 6734
 Filtering checks passed
Random state fixed at 42


Findings

The dataset has been cleaned and tokenized into meaningful words.

Words that are too rare or too frequent were removed, improving the quality of the vocabulary.

We generated two key outputs:

processed.csv → Stores the cleaned reviews.

vocab.txt → Stores the alphabetical vocabulary with indices.

This concludes Task 1. The dataset is now ready for feature extraction and model building in the next milestone.

## Summary
##### In this milestone,
##### I successfully carried out the end-to-end process of text pre-processing on a large collection of women’s clothing reviews. The goal was to transform raw, noisy review text into a structured and meaningful representation that can serve as a reliable foundation for downstream machine learning tasks.

##### Through systematic steps—tokenization, lowercasing, stopword removal, filtering out overly rare and overly frequent words—I was able to clean the dataset and reduce it to its most informative components. I also built a well-structured vocabulary file that maps each unique token to a numerical index, ensuring consistency and reproducibility for later feature extraction and model training.

##### What stood out to me in this task was how each individual step, while simple on its own, collectively contributes to the quality of the data

##### Removing tokens that are either too rare or too common helped strike a balance between capturing meaningful information and avoiding noise

##### Overall, this task gave me hands-on experience with the practical challenges of natural language preprocessing. It reinforced the importance of data cleaning as the foundation of any NLP pipeline. The processed dataset (processed.csv) and the vocabulary file (vocab.txt) I produced will directly support the feature engineering and modeling stages in the next milestone, making me more confident about tackling text classification problems in a structured, professional way.

### Overview

Accomplished: Preprocessed 19,662 reviews into clean data with 7,529-word vocabulary.
Removed:

Standard stopwords
6,734 rare words (TF=1)
20 top-frequency words (DF)

Results:

Average ~18.1 tokens per review
Vocabulary reduction: ~47% from post-stopword stage
Both output files meet format requirements

Validation: The 89.01% classification accuracy achieved in Task 3 validates these preprocessing choices—the curated vocabulary provides discriminative features for learning.
Key decisions:

Regex preserves hyphenated terms while excluding noise
TF=1 filtering removes typos/outliers
Top-20 DF filtering removes overly common domain terms
Result: focused vocabulary of meaningful, discriminative terms